In [1]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.1 MB/s eta 0:00:00


In [2]:
import os
import csv
import json
import numpy as np 
import pandas as pd 
import sqlite3
import re
import datetime
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline 

# Source to Target Mapping - OOP Approach for Single Table

> Indented block



### Data

In [25]:
with open("sales_data.csv", "r", encoding="utf-8-sig") as file:
    data = list(csv.DictReader(file))
    for item in data:
        item["InvoiceNo"] =str(item["InvoiceNo"])
        item["Quantity"] =str(item["Quantity"])
data[:3]

[{'InvoiceNo': '358640',
  'StockCode': '85123A',
  'Quantity': '6',
  'InvoiceDate': '03/01/2015 00:00',
  'CustomerID': '6858',
  'Country': 'United Kingdom'},
 {'InvoiceNo': '386946',
  'StockCode': '71053',
  'Quantity': '6',
  'InvoiceDate': '07/03/2010 00:00',
  'CustomerID': '11903',
  'Country': 'United Kingdom'},
 {'InvoiceNo': '378650',
  'StockCode': '84406B',
  'Quantity': '8',
  'InvoiceDate': '05/05/2014 00:00',
  'CustomerID': '14199',
  'Country': 'United Kingdom'}]

[link text](https://)###  Abstract Base Calss (ABC) for mutual methods

In [4]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [5]:
class TransformMask(Enum):
    CLEAN_STRING = ".strip()" 
    #date format to retrive only date without time
    DATE_FORMAT = ".split()[0]"


### Database Class - Define Common Properties for Source, Target, Mapping

In [6]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"InvoiceNo","$.InvoiceNo","str",True)
        self.add_source(2,"Quantity","$.Quantity","str",True)
        self.add_source(3,"InvoiceDate","$.InvoiceDate","str",False)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')
        self.add_transform(2,'DATE_FORMAT')


        #add_destination
        self.add_destination(1,"invoice_id","invoice_id","str","n/a")
        self.add_destination(2,"quantity","quantity","int","0")
        self.add_destination(3,"invoice_dtm","invoice_dtm","str","01/01/1900")

        #add_mapping
        self.add_mapping(1,1,1,1)
        self.add_mapping(2,2,2)
        self.add_mapping(3,3,3,2)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)    
        
    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property 
    def get_data_source_target_mapping(self):
        return self.db

### Source class

Inherited from Interface for the common methods and from Database for common variables

In [7]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

### Target class

Inherited from Interface for the common methods and from Database for common variables

In [8]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Transform Class

Inherited from Interface for the common methods and from Database for common variables

In [9]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Mapping class

Inherited from Interface for the common methods and from Database for common variables

In [10]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file 

In [11]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value



```
# This is formatted as code
```

### Combine it All - STTM

In [12]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))
               # print(mappings.get("mapping_destination"))
                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [13]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'invoice_id': '358640', 'quantity': 6, 'invoice_dtm': '03/01/2015'}
{'invoice_id': '386946', 'quantity': 6, 'invoice_dtm': '07/03/2010'}
{'invoice_id': '378650', 'quantity': 8, 'invoice_dtm': '05/05/2014'}
{'invoice_id': '70847', 'quantity': 6, 'invoice_dtm': '18/05/2021'}
{'invoice_id': '127544', 'quantity': 6, 'invoice_dtm': '22/10/2013'}
{'invoice_id': '206408', 'quantity': 2, 'invoice_dtm': '10/04/2017'}
{'invoice_id': '429117', 'quantity': 6, 'invoice_dtm': '11/04/2016'}
{'invoice_id': '409366', 'quantity': 6, 'invoice_dtm': '22/04/2022'}
{'invoice_id': '181714', 'quantity': 6, 'invoice_dtm': '15/06/2014'}
{'invoice_id': '405780', 'quantity': 32, 'invoice_dtm': '06/05/2022'}
{'invoice_id': '418234', 'quantity': 6, 'invoice_dtm': '17/10/2019'}
{'invoice_id': '381824', 'quantity': 6, 'invoice_dtm': '05/05/2017'}
{'invoice_id': '322100', 'quantity': 8, 'invoice_dtm': '03/01/2017'}
{'invoice_id': '234784', 'quantity': 6, 'invoice_dtm': '01/11/2017'}
{'invoice_id': '9162', 'quantity':

In [17]:
sales_fact = pd.DataFrame(transformed_data)

In [15]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [19]:
sales_fact.to_csv('Sales_fact.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory


Customer Dim

In [26]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"CustomerID","$.CustomerID","str",True)
        self.add_source(2,"Country","$.Country","str",True)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')


        #add_destination
        self.add_destination(1,"customer_id","customer_id","str","n/a")
        self.add_destination(2,"country","country","str","n/a")

        #add_mapping
        self.add_mapping(1,1,1)
        self.add_mapping(2,2,2,1)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)    
        
    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property 
    def get_data_source_target_mapping(self):
        return self.db

In [27]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'customer_id': '6858', 'country': 'United Kingdom'}
{'customer_id': '11903', 'country': 'United Kingdom'}
{'customer_id': '14199', 'country': 'United Kingdom'}
{'customer_id': '15258', 'country': 'United Kingdom'}
{'customer_id': '10907', 'country': 'United Kingdom'}
{'customer_id': '10624', 'country': 'United Kingdom'}
{'customer_id': '1598', 'country': 'United Kingdom'}
{'customer_id': '5434', 'country': 'United Kingdom'}
{'customer_id': '4278', 'country': 'United Kingdom'}
{'customer_id': '1703', 'country': 'United Kingdom'}
{'customer_id': '3424', 'country': 'United Kingdom'}
{'customer_id': '9661', 'country': 'United Kingdom'}
{'customer_id': '14843', 'country': 'United Kingdom'}
{'customer_id': '7437', 'country': 'United Kingdom'}
{'customer_id': '4253', 'country': 'United Kingdom'}
{'customer_id': '5316', 'country': 'United Kingdom'}
{'customer_id': '11731', 'country': 'United Kingdom'}
{'customer_id': '11468', 'country': 'United Kingdom'}
{'customer_id': '5194', 'country': 'Un

In [28]:
customer_dim = pd.DataFrame(transformed_data)

In [29]:
customer_dim.to_csv('customer_dim.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory


Product Dim

In [31]:
with open("product_data.csv", "r", encoding="utf-8-sig") as file:
    data = list(csv.DictReader(file))
    for item in data:
        item["UnitPrice"] =str(item["UnitPrice"])
data[:3]

[{'StockCode': '85123A',
  'Description': 'WHITE HANGING HEART T-LIGHT HOLDER',
  'UnitPrice': '2.55'},
 {'StockCode': '71053',
  'Description': 'WHITE METAL LANTERN',
  'UnitPrice': '3.39'},
 {'StockCode': '84406B',
  'Description': 'CREAM CUPID HEARTS COAT HANGER',
  'UnitPrice': '2.75'}]

In [32]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        #add_source
        self.add_source(1,"StockCode","$.StockCode","str",True)
        self.add_source(2,"Description","$.Description","str",True)
        self.add_source(3,"UnitPrice","$.UnitPrice","str",False)

        #add_transform
        self.add_transform(1,'CLEAN_STRING')


        #add_destination
        self.add_destination(1,"stock_code","stock_code","str","n/a")
        self.add_destination(2,"description","description","str","n/a")
        self.add_destination(3,"unit_price","unit_price","float","0.0")

        #add_mapping
        self.add_mapping(1,1,1,1)
        self.add_mapping(2,2,2,1)
        self.add_mapping(3,3,3)

    def add_source(self,id,source_field_name,source_field_mapping,source_field_type,is_required):
      if not isinstance(is_required, bool):
        print("Error. Please enter if required.")
      else:
        source = {
            "id": id,
            "source_field_name": source_field_name,
            "source_field_mapping": source_field_mapping,
            "source_field_type": source_field_type,
            "is_required": is_required
        }
        self.db["source"].append(source)

    def add_transform(self,id,transform_mask):
      transform={
          "id": id,
          "transform_mask": transform_mask
      }
      self.db["transform"].append(transform)    
        
    def add_destination(self,id,destination_field_name,destination_field_mapping,destination_field_type,default_value):
        destination = {
              "id": id,
              "destination_field_name": destination_field_name,
              "destination_field_mapping": destination_field_mapping,
              "destination_field_type": destination_field_type,
              "default_value": default_value
          }
        self.db["destination"].append(destination)

    def add_mapping(self,id,mapping_source,mapping_destination,mapping_transform=None):
      if mapping_transform is not None:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination,
          "mapping_transform": mapping_transform
        }
      else:
        mapping={
          "id": id,
          "mapping_source": mapping_source,
          "mapping_destination": mapping_destination
          }
      self.db["mapping"].append(mapping)

    @property 
    def get_data_source_target_mapping(self):
        return self.db

In [33]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'stock_code': '85123A', 'description': 'WHITE HANGING HEART T-LIGHT HOLDER', 'unit_price': 2.55}
{'stock_code': '71053', 'description': 'WHITE METAL LANTERN', 'unit_price': 3.39}
{'stock_code': '84406B', 'description': 'CREAM CUPID HEARTS COAT HANGER', 'unit_price': 2.75}
{'stock_code': '84029G', 'description': 'KNITTED UNION FLAG HOT WATER BOTTLE', 'unit_price': 3.39}
{'stock_code': '84029E', 'description': 'RED WOOLLY HOTTIE WHITE HEART.', 'unit_price': 3.39}
{'stock_code': '22752', 'description': 'SET 7 BABUSHKA NESTING BOXES', 'unit_price': 7.65}
{'stock_code': '21730', 'description': 'GLASS STAR FROSTED T-LIGHT HOLDER', 'unit_price': 4.25}
{'stock_code': '22633', 'description': 'HAND WARMER UNION JACK', 'unit_price': 1.85}
{'stock_code': '22632', 'description': 'HAND WARMER RED POLKA DOT', 'unit_price': 1.85}
{'stock_code': '84879', 'description': 'ASSORTED COLOUR BIRD ORNAMENT', 'unit_price': 1.69}
{'stock_code': '22745', 'description': "POPPY'S PLAYHOUSE BEDROOM", 'unit_price':

In [34]:
product_dim = pd.DataFrame(transformed_data)

In [35]:
product_dim.to_csv('product_dim.csv')
!cp data.csv "drive/My Drive/"

cp: cannot stat 'data.csv': No such file or directory
